## IMPORTAR LAS LIBRERIAS

Actualizar a las que se usen finalmente en tu proyecto.

In [1]:
import numpy as np
import pandas as pd
import cloudpickle

#Automcompletar rápido
%config IPCompleter.greedy=True

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

import warnings
warnings.filterwarnings("ignore")

## CARGAR LOS DATOS

### Ruta del proyecto

In [2]:
ruta_proyecto = r'C:\Users\iagzg\Desktop\D4FB\D4FB\EstructuraDirectorio\03_MACHINE_LEARNING\06-MACHINE LEARNING PROJECT FRAMEWORK\CASOS\04_BANK\\'

### Nombre del fichero de datos

In [3]:
nombre_fichero_datos = 'Bank.csv'

### Cargar los datos

In [4]:
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos

df = pd.read_csv(ruta_completa,sep = ';')

### Seleccionar solo las variables finales

#### Cargar la lista de variables finales

In [5]:
nombre_variables_finales = ruta_proyecto + '/05_Resultados/' + 'variables_finales.pickle'

pd.read_pickle(nombre_variables_finales).sort_index().index.to_list()

['canal_contacto_cellular',
 'canal_contacto_telephone',
 'dia_semana_fri',
 'dia_semana_thu',
 'edad_disc_ds_mms',
 'estado_married',
 'mes_apr',
 'mes_dec',
 'mes_jul',
 'mes_mar',
 'mes_may',
 'mes_oct',
 'prestamo_hipotecario_no',
 'prestamo_hipotecario_yes',
 'prestamo_personal_no',
 'trabajo_admin.',
 'trabajo_retired',
 'trabajo_student',
 'variacion_tasa_empleo_bin_mms']

#### Crear la matriz de variables procesos (excel)

Ir a la plantilla de Excel "Fase Producción Plantilla Procesos" y crear la matriz de variables por procesos.
Ubicado en 01_Documentos.

#### Actualizar las importaciones

Ir arriba a la celda de importacion de paquetes y actualizarlos con los que finalmente vamos a usar.

#### Variables finales

In [6]:
variables_finales = ['canal_contacto',
                     'dia_semana',
                     'edad',
                     'estado',
                     'mes',
                     'prestamo_hipotecario',
                     'trabajo',              
                     'variacion_tasa_empleo']

## ESTRUCTURA DE LOS DATASETS

### Corregir los nombres

In [7]:
df.rename(columns = {'age':'edad',
                     'job':'trabajo',
                     'marital':'estado',
                     'education':'educacion',
                     'default':'impagos',
                     'housing':'prestamo_hipotecario',
                     'loan':'prestamo_personal',
                     'contact':'canal_contacto',
                     'month':'mes',
                     'day_of_week':'dia_semana',
                     'campaign':'num_contactos_esta_semana',
                     'pdays':'num_dias_ultimo_contacto',
                     'previous':'num_contactos_otras_campañas',
                     'poutcome':'resultado_campana_anterior',
                     'emp.var.rate':'variacion_tasa_empleo'
                    }, inplace=True)

### Eliminar registros

#### Por duplicados

In [8]:
df.drop_duplicates(inplace = True)

#### Atipicos

In [9]:
var_atipicos_dt = ['edad','variacion_tasa_empleo']

###### Definir el número de desviaciones típicas

In [10]:
num_desv_tip = 4

###### Función que devuelve los índices de los atípicos

In [11]:
def atipicos_desv_tip(variable, num_desv_tip = 4):
    #sacamos los nulos por ahora
    variable = variable.dropna()
    #calculamos los límites
    media = np.mean(variable)
    sd = np.std(variable)
    umbral = sd * num_desv_tip
    lim_inf = media - umbral
    lim_sup = media + umbral
    #encontramos los índices de los que están fuera de los límites
    indices = [indice for indice,valor in variable.items() if valor < lim_inf or valor > lim_sup]
    return(indices)

###### Función que cuenta el número de atípicos

In [12]:
def conteo_atipicos(df,variable, num_desv_tip = 4):
    atipicos = atipicos_desv_tip(df[variable], num_desv_tip)
    return(df.loc[atipicos,variable].value_counts())

In [13]:
df

,edad,trabajo,estado,educacion,impagos,prestamo_hipotecario,prestamo_personal,canal_contacto,mes,dia_semana,...,num_contactos_esta_semana,num_dias_ultimo_contacto,num_contactos_otras_campañas,resultado_campana_anterior,variacion_tasa_empleo,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [14]:
for variable in var_atipicos_dt:
    df.drop(atipicos_desv_tip(df[variable], num_desv_tip),axis = 0, inplace = True)

In [15]:
df

,edad,trabajo,estado,educacion,impagos,prestamo_hipotecario,prestamo_personal,canal_contacto,mes,dia_semana,...,num_contactos_esta_semana,num_dias_ultimo_contacto,num_contactos_otras_campañas,resultado_campana_anterior,variacion_tasa_empleo,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


#### EDA 

In [16]:
df.drop('impagos', axis = 1, inplace = True)

In [17]:
df = df[(df['prestamo_hipotecario'] != 'unknown') & (df['prestamo_personal'] != 'unknown')]

#### Para x

Quedarse solo con las de la lista.

In [18]:
x = df[variables_finales].copy()

#### Para y

Especificar la target.

In [19]:
target = 'y'

Crear el y.

In [20]:
y = df[target].copy()

## CREAR EL PIPELINE

### Instanciar calidad de datos

#### Crear la función

In [21]:
def calidad_datos(df):
    temp = df.astype({'edad': 'Int64'})             
    
    return(temp)

#### Convertirla en transformer

In [22]:
hacer_calidad_datos = FunctionTransformer(calidad_datos)

### Instanciar transformación de variables

In [23]:
var_ohe = ['canal_contacto','dia_semana','estado','mes','prestamo_hipotecario','trabajo']
ohe = OneHotEncoder(sparse_output= False, handle_unknown='ignore')

var_disc_ds = ['edad']
disc_ds = KBinsDiscretizer(n_bins = 6, strategy = 'uniform', encode = 'ordinal')

var_bin = ['variacion_tasa_empleo']
bin = Binarizer(threshold=0)

var_mms = ['edad','variacion_tasa_empleo']
mms = MinMaxScaler()

### Crear el pipe del preprocesamiento

#### Crear el column transformer

In [24]:
ct = make_column_transformer(
    (ohe, var_ohe),
    (disc_ds, var_disc_ds),
    (bin, var_bin),
    (mms, var_mms),
    remainder='drop')

#### Crear el pipeline del preprocesamiento

In [25]:
pipe_prepro = make_pipeline(hacer_calidad_datos, 
                            ct)

### Instanciar el modelo

#### Instanciar el algoritmo

In [26]:
modelo = RandomForestClassifier(max_depth=5, n_estimators=200, n_jobs=-1)

#### Crear el pipe final de entrenamiento

In [27]:
pipe_entrenamiento = make_pipeline(pipe_prepro,modelo)

#### Guardar el pipe final de entrenamiento

In [28]:
nombre_pipe_entrenamiento = 'pipe_entrenamiento.pickle'

ruta_pipe_entrenamiento = ruta_proyecto + '/04_Modelos/' + nombre_pipe_entrenamiento

with open(ruta_pipe_entrenamiento, mode='wb') as file:
   cloudpickle.dump(pipe_entrenamiento, file)

#### Entrenar el pipe final de ejecución

In [29]:
pipe_ejecucion = pipe_entrenamiento.fit(x,y)

## GUARDAR EL PIPE

### Nombre del pipe final de ejecución

In [30]:
nombre_pipe_ejecucion = 'pipe_ejecucion.pickle'

### Guardar el pipe final de ejecución

In [31]:
ruta_pipe_ejecucion = ruta_proyecto + '/04_Modelos/' + nombre_pipe_ejecucion

with open(ruta_pipe_ejecucion, mode='wb') as file:
   cloudpickle.dump(pipe_ejecucion, file)